In [ ]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
    width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   width:max-content;
}
</style>

In [1]:
import operator

from pyspark.sql import Row
from pyspark.sql.functions import udf, when, rand, struct, col, lit
from pyspark.sql.types import StringType, IntegerType, StructField, StructType, DoubleType, ArrayType

In [ ]:
from enquire.vendetta import Ophelia

In [ ]:
if w is None:
    self.W = np.random.dirichlet(np.ones(5), size=1).tolist()[0]
else:
    self.W = w  # valores que pueden venir de otro origen, e.g. .conf

In [2]:
def factor_weights(**kargs):
    dict_weight = {
        "gender": float(0.0) if kargs.get("gender") is None else kargs.get("gender"),
        "age": float(0.0) if kargs.get("age") is None else kargs.get("age"),
        "education": float(0.0) if kargs.get("education") is None else kargs.get("education"),
        "job": float(0.0) if kargs.get("job") is None else kargs.get("job"),
        "marital": float(0.0) if kargs.get("marital") is None else kargs.get("marital"),
        "child": float(0.0) if kargs.get("child") is None else kargs.get("child"),
        "saving": float(0.0) if kargs.get("saving") is None else kargs.get("saving"),
        "insight": float(0.0) if kargs.get("insight") is None else kargs.get("insight"),
        "backup": float(0.0) if kargs.get("backup") is None else kargs.get("backup")
    }
    
    return dict_weight

In [3]:
W = factor_weights(
    gender=0.0072,
    age=0.3205,
    education=0.0009,
    job=0.3905,
    marital=0.0024,
    child=0.1404,
    saving=0.0423,
    insight=0.0335,
    backup=0.0623
)

W

{'gender': 0.0072,
 'age': 0.3205,
 'education': 0.0009,
 'job': 0.3905,
 'marital': 0.0024,
 'child': 0.1404,
 'saving': 0.0423,
 'insight': 0.0335,
 'backup': 0.0623}

In [ ]:
ophelia = Ophelia()

In [ ]:
#spark = ophelia.spk
#customer_banking = ophelia.tr.read_file("data/raw/csv/bank.csv", "csv", spark)
#customer_banking.show(5)

In [ ]:
schema_tree = ArrayType(StructType([StructField("weight", DoubleType(), True),
                                    StructField("risk_label", StringType(), True),
                                    StructField("vote", IntegerType(), True)]))

In [ ]:
tree_udf = udf(ophelia.clss.tree_generator, schema_tree)
classification_udf = udf(ophelia.clss.run_classification_risk, StringType())

In [ ]:
#random_sample = customer_banking\
#    .select("age", "job", "marital", "education")\
#    .withColumn("gender", when(rand() > 0.5, "male").otherwise("female"))\
#    .withColumn("child", when(rand() > 0.5, 1).otherwise(0))\
#    .withColumn("saving", when(rand() > 0.5, 1).otherwise(0))\
#    .withColumn("insight", when(rand() > 0.5, 1).otherwise(0))\
#    .withColumn("backup", when(rand() > 0.5, 1).otherwise(0))\
#    .withColumn("struct", struct(col("gender"), col("age"), col("education"), col("job"), 
#                                 col("marital"), col("child"), col("saving"), col("insight"), col("backup")))
#random_sample.show(5)

In [ ]:
#path = "data/master/ophelia/data/OpheliaData/risk_classification_customer/"
#random_sample.repartition(int(1)).write.mode("overwrite").parquet(path)

# a partir de aqui

In [ ]:
spark = ophelia.spk
path = "data/master/ophelia/data/OpheliaData/risk_classification/"
random_sample_df = spark.read.parquet(path)
random_sample_df.show(5)

In [ ]:
#spark = ophelia.spk
#path = "data/master/ophelia/data/OpheliaData/risk_classification_customer/"
#random_sample_df = spark.read.parquet(path)
#random_sample_df.show(5)

In [ ]:
#schema_tree = ArrayType(StructType([StructField("vote", IntegerType(), True),
#                                    StructField("weight", DoubleType(), True),
#                                    StructField("risk_label", StringType(), True)]))
#
#tree_udf = udf(tree_generator, schema_tree)
#classification_udf = udf(run_classification_risk, StringType())

In [ ]:
result_df = random_sample_df.select("*", tree_udf(col("struct")).alias("tree")).cache()
result_df.show(5)

In [ ]:
final = result_df.select("age", "job", "marital", "education", "gender",
                         "child", "saving", "insight", "backup",
                         classification_udf(col("tree")).alias("risk_label"))

final.groupBy("risk_label").count().show(20, False)
final.show(10)

In [ ]:
final.where((col("risk_label") == "A") & (col("gender") == "female")).orderBy(col("age").desc()).show(1000000)

In [14]:
score_gender("female", W)

Row(vote=4, weight=0.0072, risk_label='MA')

In [28]:
score_education("primary", W)

Row(vote=1, weight=0.0009, risk_label='MC')

In [33]:
score_occupation("retired", W)

Row(vote=10, weight=0.3905, risk_label='C')

In [29]:
+-------------+
|          job|
+-------------+
|   management|
|      retired| 1
|      unknown| 1
|self-employed| 1
|      student| 1
|  blue-collar| 1
| entrepreneur| 1
|       admin.| 1
|   technician| 1
|     services| 1
|    housemaid| 1
|   unemployed| 1
+-------------+

TypeError: unhashable type: 'list'

In [30]:
# Refactor to hash-maps

def score_gender(gender, w):
    vote = 1
    w_i = w["gender"]
    max_vote = len(w)/2
    hash_map_option = {
        "female": key_value(max_vote, w_i, "MA"),
        "male": key_value(vote, w_i, "A")
    }
    return hash_map_option[gender]

def score_education(education, w):
    vote = 1
    w_i = w["education"]
    try:
        hash_map_option = {
            "unknown": key_value(vote, w_i, "C"),
            "primary": key_value(vote, w_i, "MC"),
            "secondary": key_value(vote, w_i, "MA"),
            "tertiary": key_value(vote, w_i, "A")
        }
        return hash_map_option[education]
    except KeyError:
        raise KeyError("no se puede chavo")
        
def score_occupation(occupation, w):
    vote = 1
    w_i = w["job"]
    min_vote = len(w)/2
    mid_vote = min_vote+(len(w)/5)
    max_vote = len(w)+1
    hash_map_option = {
        "unemployed": key_value(max_vote, w_i, "C"),
        "retired": key_value(max_vote, w_i, "C"),
        "unknown": key_value(max_vote, w_i, "C"),
        "housemaid": key_value(mid_vote, w_i, "MC"),
        "student": key_value(mid_vote, w_i, "MC"),
        "self-employed": key_value(min_vote, w_i, "M"),
        "entrepreneur": key_value(min_vote, w_i, "M"),
        "admin.": key_value(vote, w_i, "MA"),
        "technician": key_value(vote, w_i, "MA"),
        "services": key_value(vote, w_i, "MA"),
        "blue-collar": key_value(vote, w_i, "MA"),
        "white-collar": key_value(vote, w_i, "A"),
        "management": key_value(vote, w_i, "A"),
    }
    return hash_map_option[occupation]

def score_marital(marital, w):
    vote = 1
    w_i = w["marital"]
    min_vote = len(w)/5
    max_vote = len(w)/4
    hash_map_option = {
        "divorced": key_value(max_vote, w_i, "MC"),
        "married": key_value(min_vote, w_i, "MA"),
        "single": key_value(vote, w_i, "A"),
    }
    return hash_map_option[marital]

def __binary_hashmap(node, vote, w, label):
    hasmap_option = {
        "1": key_value(vote, w, label),
        "0": key_value(vote, w, label)
    }
    return hasmap_option[node]

def score_child(child, w):
    vote = 1
    w_i = w["child"]
    max_vote = len(w)/4
    
    if child == 1:
        return key_value(max_vote, w_i, "MC")
    else:
        return key_value(vote, w_i, "MA")
    
def score_saving(saving, w):
    
    vote = 1
    w_i = w["saving"]
    max_vote = len(w)/2
    
    if saving == 1:
        return key_value(max_vote, w_i, "MC")
    else:
        return key_value(vote, w_i, "MA")
    
def score_insight(insight, w):
    
    vote = 1
    w_i = w["insight"]
    max_vote = len(w)/5
    
    if insight == 1:
        return key_value(max_vote, w_i, "A")
    else:
        return key_value(vote, w_i, "M")
    
def score_backup(backup, w):
    
    vote = 1
    w_i = w["backup"]
    max_vote = len(w)/2
    
    if backup == 1:
        return key_value(vote, w_i, "MA")
    else:
        return key_value(vote, w_i, "M")

In [9]:
def key_value(vote, w, clss):
    return Row(vote=round(float(vote)), weight=float(w), risk_label=str(clss))

def score_gender(gender, w):
    
    vote = 1
    w_i = w["gender"]
    max_vote = len(w)/2
    
    if gender in ["female"]:
        return key_value(max_vote, w_i, "MA")
    elif gender in ["male"]:
        return key_value(vote, w_i, "A")

def score_age(age, w):
    
    vote = 1
    w_i = w["age"]
    min_vote = len(w)/2
    mid_vote = min_vote+(len(w)/5)
    max_vote = len(w)+1
    
    def two_bound(age, min_bound, upper_bound):
        if min_bound <= age < upper_bound:
            return age
       
    if two_bound(age, min_bound=60, upper_bound=100):
        if two_bound(age, min_bound=67, upper_bound=100):
            return key_value(max_vote, w_i, "C")
        elif two_bound(age, min_bound=60, upper_bound=67):
            return key_value(max_vote, w_i, "MC")
    
    elif two_bound(age, min_bound=50, upper_bound=60):
        if two_bound(age, min_bound=57, upper_bound=60):
            return key_value(mid_vote, w_i, "MC")
        elif two_bound(age, min_bound=50, upper_bound=57):
            return key_value(mid_vote, w_i, "M")
    
    elif two_bound(age, min_bound=40, upper_bound=50):
        if two_bound(age, min_bound=47, upper_bound=50):
            return key_value(min_vote, w_i, "M")
        elif two_bound(age, min_bound=40, upper_bound=47):
            return key_value(min_vote, w_i, "MA")
    
    elif two_bound(age, min_bound=30, upper_bound=40):
        if two_bound(age, min_bound=37, upper_bound=40):
            return key_value(vote, w_i, "MA")
        elif two_bound(age, min_bound=30, upper_bound=37):
            return key_value(vote, w_i, "A")
    else:
        return key_value(vote, w_i, "A")
    

def score_education(education, w):
    
    vote = 1
    w_i = w["education"]
    
    unknown = ["unknown"]
    middle = ["primary"]
    university = ["secondary"]
    master = ["tertiary"]
    
    if education in unknown:
        return key_value(vote, w_i, "C")
    elif education in middle:
        return key_value(vote, w_i, "MC")
    elif education in university:
        return key_value(vote, w_i, "MA")
    elif education in master:
        return key_value(vote, w_i, "A")


def score_occupation(occupation, w):

    vote = 1
    w_i = w["job"]
    min_vote = len(w)/2
    mid_vote = min_vote+(len(w)/5)
    max_vote = len(w)+1
    
    unemployed = ["unemployed", "retired", "unknown"]
    no_employee = ["housemaid", "student"]
    self_employee = ["self-employed", "entrepreneur"]
    employee = ["admin.", "technician", "services", "blue-collar"]

    if occupation in unemployed:
        return key_value(max_vote, w_i, "C")
    elif occupation in no_employee:
        return key_value(mid_vote, w_i, "MC")
    elif occupation in self_employee:
        return key_value(min_vote, w_i, "M")
    elif occupation in employee:
        return key_value(vote, w_i, "MA")
    else:
        return key_value(vote, w_i, "A")
    
def score_marital(marital, w):
    
    vote = 1
    w_i = w["marital"]
    min_vote = len(w)/5
    max_vote = len(w)/4
    
    if marital in ["married"]:
        return key_value(min_vote, w_i, "MA")
    elif marital in ["single"]:
        return key_value(vote, w_i, "A")
    elif marital in ["divorced"]:
        return key_value(max_vote, w_i, "MC")

def score_child(child, w):
    
    vote = 1
    w_i = w["child"]
    max_vote = len(w)/4
    
    if child == 1:
        return key_value(max_vote, w_i, "MC")
    else:
        return key_value(vote, w_i, "MA")
    
def score_saving(saving, w):
    
    vote = 1
    w_i = w["saving"]
    max_vote = len(w)/2
    
    if saving == 1:
        return key_value(max_vote, w_i, "MC")
    else:
        return key_value(vote, w_i, "MA")
    
def score_insight(insight, w):
    
    vote = 1
    w_i = w["insight"]
    max_vote = len(w)/5
    
    if insight == 1:
        return key_value(max_vote, w_i, "A")
    else:
        return key_value(vote, w_i, "M")
    
def score_backup(backup, w):
    
    vote = 1
    w_i = w["backup"]
    max_vote = len(w)/2
    
    if backup == 1:
        return key_value(vote, w_i, "MA")
    else:
        return key_value(vote, w_i, "M")

# class 2

In [ ]:
def factor_weights(**kargs):
    
    dict_weight = {
        "gender": float(0.0) if kargs.get("gender") is None else float(kargs.get("gender")),
        "age": float(0.0) if kargs.get("age") is None else float(kargs.get("age")),
        "education": float(0.0) if kargs.get("education") is None else float(kargs.get("education")),
        "job": float(0.0) if kargs.get("job") is None else float(kargs.get("job")),
        "marital": float(0.0) if kargs.get("marital") is None else float(kargs.get("marital")),
        "child": float(0.0) if kargs.get("child") is None else float(kargs.get("child")),
        "saving": float(0.0) if kargs.get("saving") is None else float(kargs.get("saving")),
        "insight": float(0.0) if kargs.get("insight") is None else float(kargs.get("insight")),
        "backup": float(0.0) if kargs.get("backup") is None else float(kargs.get("backup"))
    }
    
    return dict_weight

def tree_generator(obj):

    w = factor_weights(
        gender=0.0072,
        age=0.3205,
        education=0.0009,
        job=0.3905,
        marital=0.0024,
        child=0.1404,
        saving=0.0423,
        insight=0.0335,
        backup=0.0623
    )

    tree = [
        score_gender(gender=obj.gender, w=w),
        score_age(age=obj.age, w=w),
        score_education(education=obj.education, w=w),
        score_occupation(occupation=obj.job, w=w),
        score_marital(marital=obj.marital, w=w),
        score_child(child=obj.child, w=w),
        score_saving(saving=obj.saving, w=w),
        score_insight(insight=obj.insight, w=w),
        score_backup(backup=obj.backup, w=w)
    ]

    return tree

def dict_filter(dic):
    filter_dic = {}
    for item in dic:
        if dic[item] > 0:
            filter_dic[item] = dic[item]
    return filter_dic

def group_key(a, ma, m, mc, c, counter=0.0):

    group_key_value = {"A": float(a), "MA": float(ma), "M": float(m), "MC": float(mc), "C": float(c), "tot": float(counter)}

    return group_key_value

def group_by_key(tree, key, a=0.0, ma=0.0, m=0.0, mc=0.0, c=0.0, counter=0.0):
    
    for item in tree:
        counter += tree.count(item) 
        if item["risk_label"] == "A":
            a += item[key]
        elif item["risk_label"] == "MA":
            ma += item[key]
        elif item["risk_label"] == "M":
            m += item[key]
        elif item["risk_label"] == "MC":
            mc += item[key]
        elif item["risk_label"] == "C":
            c += item[key]
    
    return dict_filter(group_key(a, ma, m, mc, c, counter))

def prob_label(dic, a=0.0, ma=0.0, m=0.0, mc=0.0, c=0.0):
    
    for item in dic:
        if item == "A":
            a = dic[item] / dic["tot"]
        elif item == "MA":
            ma = dic[item] / dic["tot"]
        elif item == "M":
            m = dic[item] / dic["tot"]
        elif item == "MC":
            mc = dic[item] / dic["tot"]
        elif item == "C":
            c = dic[item] / dic["tot"]
    
    return dict_filter(group_key(a, ma, m, mc, c))


def matmul_dict(weight, prob, w_a=0.0, w_ma=0.0, w_m=0.0, w_mc=0.0, w_c=0.0):
    
    for item in weight:
        if item == "A":
            w_a = (weight[item] * prob[item]) + weight[item]
        elif item == "MA":
            w_ma = (weight[item] * prob[item]) + weight[item]
        elif item == "M":
            w_m = (weight[item] * prob[item]) + weight[item]
        elif item == "MC":
            w_mc = (weight[item] * prob[item]) + weight[item]
        elif item == "C":
            w_c = (weight[item] * prob[item]) + weight[item]

    return dict_filter(group_key(w_a, w_ma, w_m, w_mc, w_c))


def consensus(dot, freq, threshold=0.6, truncate=0.5, w_a=0.0, w_ma=0.0, w_m=0.0, w_mc=0.0, w_c=0.0):

    def truncate(x, thr=threshold, trunc=truncate):
        return round(x - thr + trunc)
    
    for item in dot:
        if item == "A":
            w_a = truncate(dot[item] + freq[item])
        elif item == "MA":
            w_ma = truncate(dot[item] + freq[item])
        elif item == "M":
            w_m = truncate(dot[item] + freq[item])
        elif item == "MC":
            w_mc = truncate(dot[item] + freq[item])
        elif item == "C":
            w_c = truncate(dot[item] + freq[item])

    return dict_filter(group_key(w_a, w_ma, w_m, w_mc, w_c))


def validate_values(dic):
    equal_validation = len(dic.values()) == len(set(dic.values()))
    if equal_validation is False:
        return str(1)
    return str(0)


def assign_label(dic, dot_dic):
    if len(dic) is 0:
        return str("null")
    if validate_values(dic) is "1":
        return str(max(dot_dic.items(), key=operator.itemgetter(1))[0])
    else:    
        return str(max(dic.items(), key=operator.itemgetter(1))[0])

    
def run_classification_risk(tree):
    aggregate_weight = group_by_key(tree, "weight")
    label_frequency = group_by_key(tree, "vote")
    label_probability = prob_label(label_frequency)
    dot = matmul_dict(aggregate_weight, label_probability)
    consensus_vote = consensus(dot, label_frequency)
    return str(assign_label(consensus_vote, dot))

In [ ]:
class Person:
    def __init__(self, gender, age, education, job, marital, child, saving, insight, backup):
        self.gender = gender
        self.age = age
        self.education = education
        self.job = job
        self.marital = marital
        self.child = child
        self.saving = saving
        self.insight = insight
        self.backup = backup

In [ ]:
luis = Person(
    gender="male",
    age=30,
    education="secondary",
    job="admin.",
    marital="married",
    child=0,
    saving=1,
    insight=1,
    backup=0
)

In [ ]:
tree = ophelia.ophelia_class.tree_generator(luis)
tree

In [ ]:
ophelia.ophelia_class.run_classification_risk(tree)